In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
import datetime

In [ ]:
from kaggle_secrets import UserSecretsClient
import wandb
from wandb.keras import WandbCallback
user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb-api-key") 
wandb.login(key=wandb_api)

In [ ]:
train_dir= '../input/plant-pathology-2021-fgvc8/train_images'
test_dir =  '../input/plant-pathology-2021-fgvc8/test_images'
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [ ]:
train = pd.DataFrame(train,columns = ['image','labels'])

In [ ]:
train['labels'].value_counts()

In [ ]:
plt.figure(figsize=(20,12))
labels = sns.barplot(train.labels.value_counts().index,train.labels.value_counts())
for item in labels.get_xticklabels():
    item.set_rotation(45)

In [ ]:
train['labels'] = train['labels'].apply(lambda s: s.split(' '))
train[:10]

In [ ]:
datagen = ImageDataGenerator(
    rotation_range = 10,#Performing Rotation
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    brightness_range = None,
    shear_range = 0.1,
    zoom_range = 0.1,
    rescale = 1./255,
    horizontal_flip=True,
    vertical_flip=True,
    
    validation_split= 0.15# We will split the training data into training and validation set in the ration 90:10
)
bsize  = 32 

In [ ]:
train_data = datagen.flow_from_dataframe(
    train,
    directory = '../input/resized-plant2021/img_sz_512',# We are using the resized images otherwise it will take a lot of time to train 
    x_col = 'image',
    y_col = 'labels',
    subset="training",
    color_mode="rgb",
    target_size = (224,224),
    class_mode="categorical",
    batch_size=bsize,
    shuffle=False,
    seed=40,
)

In [ ]:
valid_data = datagen.flow_from_dataframe(
    train,
    directory = '../input/resized-plant2021/img_sz_512',
    x_col = 'image',
    y_col = 'labels',
    subset="validation",
    color_mode="rgb",
    target_size = (224,224),
    class_mode="categorical",
    batch_size=bsize,
    shuffle=False,
    seed=40,
)

In [ ]:
from tensorflow.keras.preprocessing import image
import random
input_shape= (224,224,3)
base_model = tf.keras.applications.DenseNet121(input_shape=input_shape, include_top=False,weights= "imagenet")

In [ ]:
from tensorflow.keras.layers import MaxPooling2D,GlobalAveragePooling2D,BatchNormalization,Activation
x = base_model.output
x = GlobalAveragePooling2D()(x)
#fully connected layer
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
# finally, the softmax for the classifier 
predictions = tf.keras.layers.Dense(6, activation='sigmoid')(x)

In [ ]:
import tensorflow_addons as tfa
model = tf.keras.Model(inputs=base_model.input ,outputs = predictions)
threshold_x=0.25
optimizer = tf.keras.optimizers.Adam()
f1 = tfa.metrics.F1Score(num_classes=6, average='macro',threshold = threshold_x)

In [ ]:
#callback functions

lrschedule = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                 factor=0.05, patience=5, verbose=1)

earlyStopping = tf.keras.callbacks.EarlyStopping(monitor=f1, patience=3, mode='max', restore_best_weights=True)

checkpoint_path = "training/xception-cp-{epoch:04d}.ckpt"

checkpointCallback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch')

In [ ]:
CONFIG = dict (
    num_labels = 6,
    train_val_split = 0.15,
    img_width = 224,
    img_height = 224,
    batch_size = 32,
    epochs = 15,
    learning_rate = 0.001,
    architecture = "CNN",
    infra = "Kaggle",
    model_name = "DenseNet121"
)
CONFIG['model_name'] = 'DenseNet121'
print('Training configuration: ', CONFIG)

# Initialize W&B run
run = wandb.init(project='plant-pathology', 
                 config=CONFIG,
                 group='DenseNet121', 
                 job_type='train')


callbacks_list = [earlyStopping,lrschedule, checkpointCallback, WandbCallback()]


wandb.config.type = 'baseline'
wandb.config.kaggle_competition = 'Plant Pathology 2021 - FGVC8'

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=optimizer,metrics=[f1])

In [ ]:
history = model.fit(train_data, validation_data= valid_data, callbacks=callbacks_list, epochs = 15) 
run.finish()

In [ ]:
!mkdir -p save
model.save('save/pp_research02(DenseNet121).h5')

In [ ]:
loss, f1score = model.evaluate_generator(valid_data,verbose=1)